In [85]:
import altair as alt
import pandas as pd
import mysql.connector
from mysql.connector import Error

def conditions(kekina):
    """if((kekina['yTlak'] <= 130) and (kekina['xTlak'] <= 85)):
        return "Normal Blood Pressure"
    elif((kekina['yTlak'] in range(131,139)) and (kekina['xTlak'] in range(86,89))):
        return "Elevated Normal Pressure"
    elif((kekina['yTlak'] in range(140,159)) and (kekina['xTlak'] in range(90,99))):
        return "Mild Hypertension"
    elif((kekina['yTlak'] in range(160,179)) and (kekina['xTlak'] in range(100,109))):
        return "Moderate Hypertension"
    elif((kekina['yTlak'] > 180) and (kekina['xTlak'] > 110)):
        return "Severe Hypertension"
    elif((kekina['yTlak'] > 140) and (kekina['xTlak'] < 90)):
        return "Isolated systolic hypertension"
        """
    if((kekina['yTlak'] in range(90,119)) and (kekina['xTlak'] in range (60,89))):
        return "Normal Blood Pressure"
    elif((kekina['yTlak'] in range(120,139)) and (kekina['xTlak'] in range (80,89))):
        return "Moderate Blood Pressure"
    elif((kekina['yTlak'] > 140) or (kekina['xTlak'] > 90)):
        return "High Blood Pressure"
    else:
        return "Low Blood Pressure"


try:
    connection = mysql.connector.connect(host = "localhost", database = "ehr", user = "root", password = "")

    

    mySql_select__Query = "SELECT age,sex,bloodPressure FROM patients"
    cursor = connection.cursor(dictionary = True)
    cursor.execute(mySql_select__Query)
    row_count = 1000
    records = cursor.fetchmany(row_count)
    print("Total number of rows: ", cursor.rowcount)

    df = pd.DataFrame(records)
    bins = [0, 24, 64, 120]
    labels = ['adolescent', 'adult', 'old age']
    df['AgeGroup'] = pd.cut(df['age'], bins=bins, labels=labels, right = False)
    kekina = df['bloodPressure'].str.split('/', expand = True).astype(int)
    kekina.rename(columns = {0: 'yTlak', 1: 'xTlak'}, inplace = True)
    #kekina =kekina.squeeze()
    #print (kekina)
    #print (type(kekina))
    #pd.to_numeric(kekina)
    #kekina['bloodPressureGroup'] = pd.cut((kekina['yTlak']), bins = yTlak, labels = yTlakLabel, right = False)
    #print (kekina)
    kekina['Type'] = kekina.apply(conditions, axis=1)
    result = df.join(kekina)
    result.to_csv('file1.csv')
    #print (result)

    """normalni: prvi je ispod 120 a drugi je ispod 80, prehypertension: prvi je 120 do 139 ili drugi je od 80 do 89, """
    
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySql connection is closed")

bars = alt.Chart().mark_bar().encode(
    x='sex:N',
    y=alt.Y('mean(xTlak):Q', title='Mean xTlak (mmHG)'),
    color='sex:N',
).interactive()

alt.layer(bars, data=result).facet(
    column='AgeGroup:N'
)

Total number of rows:  1000
MySql connection is closed


alt.FacetChart(...)